## **ECG Diagnosis Code**

This code is based on the code developed here: https://doi.org/10.1038/s41467-020-15432-4

**Define Libraries**

In [1]:
from tensorflow.keras.layers import (
    Input, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, Add, Flatten, Dense)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, ReduceLROnPlateau,
                                        CSVLogger, EarlyStopping)
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, f1_score, recall_score, classification_report, confusion_matrix
import numpy as np
import h5py
import math
import pandas as pd
from tensorflow.keras.utils import Sequence
import numpy as np
import os
import pickle

In [2]:
cwd = os.getcwd()

**Load the data**

First Load in the test data

In [15]:
#Load in test data
path_to_hdf5 = cwd + '\\data\\test\\ecg_tracings.hdf5'
dataset_name = 'tracings'
path_to_csv = cwd + '\\data\\test\\gold_standard.csv'

#Order is based on test set csv
abnormalities = ['1dAVb','RBBB', 'LBBB', 'SB',  'AF', 'ST']

labels_test = pd.read_csv(path_to_csv).values
f = h5py.File(path_to_hdf5, "r")
tracings_test = f[dataset_name][()]
f.close()

Next find the class percentages in the test data since we know it was properly balanced

This can be used to select the train set

In [18]:
def FindPercents(labels:np.ndarray, average: bool=True):
    bias = []
    for ii in range(labels.shape[-1]):
        bias.append(np.sum(labels[:,ii])/len(labels[:,ii]))
    
    if average:
        bias = np.mean(bias)
        return bias
    else:
        return np.array(bias)

test_per = FindPercents(labels_test)

Now due to memory constraints can only take 3 exams from the 15% of the train set

Therefore I will take the 3 exams with the average percents closest to the balanced test set

In [22]:
def ProcessLabels(data_info: pd.DataFrame, train_ids: np.ndarray, abnormalities: list):
    good_rows = []
    bad_rows = []
    labels = np.array([])
    for ii, id in enumerate(train_ids):
        if id in data_info['exam_id'].to_numpy():
            good_rows.append(ii)
            data_row = data_info.iloc[np.where(data_info['exam_id'].to_numpy() == id)[0][0]]
            labels = np.append(labels, data_row[abnormalities].values.tolist())
        else:
            bad_rows.append(ii)

    labels = np.reshape(labels, (len(good_rows), len(abnormalities)))
    
    return labels.astype(int), np.array(good_rows)

#Load in training data - can only take 3 due to memory constraints so use the best 3
path_to_hdf5s = cwd + '\\data\\train\\exams\\'
path_to_csv = cwd + '\\data\\train\\exams.csv'
data_info = pd.read_csv(path_to_csv)
all_exams = os.listdir(path_to_hdf5s)

#Process the exams data
for ii, path_to_hdf5 in enumerate(all_exams):
    print('Loading File: {}'.format(ii))
    f = h5py.File(path_to_hdf5s + path_to_hdf5, "r")
    train_ids = f['exam_id'][()]
    f.close()
    exam_labels, good_rows = ProcessLabels(data_info, train_ids, abnormalities)
    
    #Find best exams
    if ii == 0:
        scores = np.array([np.abs(test_per-FindPercents(exam_labels))])
    else:
        scores = np.append(scores, np.abs(test_per-FindPercents(exam_labels)))

#Find the best exams
best_exams = []
for ii in range(3):
    best_exams.append(np.argmin(scores))
    scores = np.delete(scores,np.argmin(scores))

print('The best exams are: {}'.format(best_exams))

Loading File: 0
Loading File: 1
Loading File: 2
Loading File: 3
Loading File: 4
Loading File: 5
Loading File: 6
Loading File: 7
Loading File: 8
Loading File: 9
Loading File: 10
Loading File: 11
Loading File: 12
Loading File: 13
Loading File: 14
Loading File: 15
Loading File: 16
The best exams are: [9, 6, 4]


After find the best exams, load only these exams for the training data

In [23]:
#Load in training data 
path_to_hdf5s = cwd + '\\data\\train\\Best_Exams\\'
path_to_csv = cwd + '\\data\\train\\exams.csv'
data_info = pd.read_csv(path_to_csv)
all_exams = os.listdir(path_to_hdf5s)

#Process the exams data
for ii, path_to_hdf5 in enumerate(all_exams):
    print('Loading File: {}'.format(ii))
    f = h5py.File(path_to_hdf5s + path_to_hdf5, "r")
    exam_tracings = f['tracings'][()]
    train_ids = f['exam_id'][()]
    f.close()
    exam_labels, good_rows = ProcessLabels(data_info, train_ids, abnormalities)

    if ii == 0:
        tracings_train = exam_tracings[good_rows].astype(np.float16)
        labels_train = exam_labels
    else:
        tracings_train = np.append(tracings_train, exam_tracings[good_rows].astype(np.float16), axis = 0)
        labels_train = np.append(labels_train, exam_labels, axis = 0)

Loading File: 0
Loading File: 1
Loading File: 2


**Saving Data Functions**

A function for saving data

In [24]:
def SaveObject(object_to_save, save_path):
    '''Load path must have the file name with the .pickle extension'''
    pickle_out = open(save_path, 'wb')
    pickle.dump(object_to_save, pickle_out)
    pickle_out.close()

Function for loading data

In [25]:
def LoadObject(load_path):
    infile = open(load_path, 'rb')
    Loaded_object = pickle.load(infile)
    infile.close()

    return Loaded_object

## Current Model

**Define the NN model**

In [26]:
class ResidualUnit(object):
    def __init__(self, n_samples_out, n_filters_out, kernel_initializer='he_normal',
                 dropout_keep_prob=0.8, kernel_size=17, preactivation=True,
                 postactivation_bn=False, activation_function='relu'):
        self.n_samples_out = n_samples_out
        self.n_filters_out = n_filters_out
        self.kernel_initializer = kernel_initializer
        self.dropout_rate = 1 - dropout_keep_prob
        self.kernel_size = kernel_size
        self.preactivation = preactivation
        self.postactivation_bn = postactivation_bn
        self.activation_function = activation_function

    def _skip_connection(self, y, downsample, n_filters_in):
        """Implement skip connection."""
        # Deal with downsampling
        if downsample > 1:
            y = MaxPooling1D(downsample, strides=downsample, padding='same')(y)
        elif downsample == 1:
            y = y
        else:
            raise ValueError("Number of samples should always decrease.")
        # Deal with n_filters dimension increase
        if n_filters_in != self.n_filters_out:
            # This is one of the two alternatives presented in ResNet paper
            # Other option is to just fill the matrix with zeros.
            y = Conv1D(self.n_filters_out, 1, padding='same',
                       use_bias=False, kernel_initializer=self.kernel_initializer)(y)
        return y

    def _batch_norm_plus_activation(self, x):
        if self.postactivation_bn:
            x = Activation(self.activation_function)(x)
            x = BatchNormalization(center=False, scale=False)(x)
        else:
            x = BatchNormalization()(x)
            x = Activation(self.activation_function)(x)
        return x

    def __call__(self, inputs):
        """Residual unit."""
        x, y = inputs
        n_samples_in = y.shape[1]
        downsample = n_samples_in // self.n_samples_out
        n_filters_in = y.shape[2]
        y = self._skip_connection(y, downsample, n_filters_in)
        # 1st layer
        x = Conv1D(self.n_filters_out, self.kernel_size, padding='same',
                   use_bias=False, kernel_initializer=self.kernel_initializer)(x)
        x = self._batch_norm_plus_activation(x)
        if self.dropout_rate > 0:
            x = Dropout(self.dropout_rate)(x)

        # 2nd layer
        x = Conv1D(self.n_filters_out, self.kernel_size, strides=downsample,
                   padding='same', use_bias=False,
                   kernel_initializer=self.kernel_initializer)(x)
        if self.preactivation:
            x = Add()([x, y])  # Sum skip connection and main connection
            y = x
            x = self._batch_norm_plus_activation(x)
            if self.dropout_rate > 0:
                x = Dropout(self.dropout_rate)(x)
        else:
            x = BatchNormalization()(x)
            x = Add()([x, y])  # Sum skip connection and main connection
            x = Activation(self.activation_function)(x)
            if self.dropout_rate > 0:
                x = Dropout(self.dropout_rate)(x)
            y = x
        return [x, y]


def get_model(n_classes, last_layer='sigmoid'):
    kernel_size = 16
    kernel_initializer = 'he_normal'
    signal = Input(shape=(4096, 12), dtype=np.float32, name='signal')
    x = signal
    x = Conv1D(64, kernel_size, padding='same', use_bias=False,
               kernel_initializer=kernel_initializer)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x, y = ResidualUnit(1024, 128, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, x])
    x, y = ResidualUnit(256, 196, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x, y = ResidualUnit(64, 256, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x, _ = ResidualUnit(16, 320, kernel_size=kernel_size,
                        kernel_initializer=kernel_initializer)([x, y])
    x = Flatten()(x)
    diagn = Dense(n_classes, activation=last_layer, kernel_initializer=kernel_initializer)(x)
    model = Model(signal, diagn)
    return model


**Load Parameters**

Loading the parameters for the model that were found in the paper. We will call this our first model

They trained 10 NN with different initializations. The choose the model to use based on the median micro average persion (mAP = 0.951). They had to choose the one right above the median since 10 is even so they can't take the median execution

In [27]:
path_to_model = cwd + '\\model\\model.hdf5'

loss = 'binary_crossentropy'
lr = 0.001
batch_size = 64
opt = Adam(lr)

model_1 = load_model(path_to_model, compile=False)
model_1.compile(loss=loss, optimizer=Adam())

**Data Fromatting**

Here is the class for transforming the data into the proper format

In [28]:
class ECGSequence(Sequence):
    @classmethod
    def get_train_and_val(cls, tracings: np.ndarray, labels: np.ndarray=None, batch_size=8, val_split=0.02):
        n_samples = tracings.shape[0]
        n_train = math.ceil(n_samples*(1-val_split))
        train_seq = cls(tracings, labels, batch_size, end_idx=n_train)
        valid_seq = cls(tracings, labels, batch_size, start_idx=n_train)
        return train_seq, valid_seq

    def __init__(self, tracings:np.ndarray, labels:np.ndarray=None, batch_size:int=8,\
        start_idx=0, end_idx=None):
        if labels is None:
            self.y = None
        else:
            self.y = labels
        # Get tracings
        self.x = tracings
        self.batch_size = batch_size
        if end_idx is None:
            end_idx = len(self.x)
        self.start_idx = start_idx
        self.end_idx = end_idx

    @property
    def n_classes(self):
        return self.y.shape[1]

    def __getitem__(self, idx):
        start = self.start_idx + idx * self.batch_size
        end = min(start + self.batch_size, self.end_idx)
        if self.y is None:
            return np.array(self.x[start:end, :, :])
        else:
            return np.array(self.x[start:end, :, :]), np.array(self.y[start:end])

    def __len__(self):
        return math.ceil((self.end_idx - self.start_idx) / self.batch_size)


**Training Function**

We will also train the model with the data accessible for better comparison with the simplified model. We will call this the second model

For sake of computational resources and time, the second model was only trained once instead of trained 10 times and then taking the model based on the median mAP

In [29]:
class MyCNN:
    def __init__(self, loss, opt, verbose, save_path=None):
        # Optimization settings
        self.callbacks = [ReduceLROnPlateau(monitor='val_loss',
                            factor=0.1,
                            patience=7,
                            min_lr=lr / 100),
                            EarlyStopping(monitor='val_loss', 
                            patience=9,  # Patience should be larger than the one in ReduceLROnPlateau
                            min_delta=0.00001)]

        self.loss = loss
        self.optimizer = opt
        self.verbose = verbose
        
        # Save the BEST and LAST model
        if not save_path is None:
            self.callbacks += [ModelCheckpoint(save_path, save_best_only=True)]

    def train(self, train_seq, val_seq):
        self.model = get_model(train_seq.n_classes)
        self.model.compile(loss=self.loss, optimizer=self.optimizer)
        # Train neural network
        self.model.fit(train_seq,
            epochs=50,
            initial_epoch=0,  # If you are continuing a interrupted section change here
            callbacks=self.callbacks,
            validation_data=val_seq,
            verbose=self.verbose)

    def predict(self, test_seq):
        return self.model.predict(test_seq,  verbose=self.verbose)

## Simplified Models

Need to choose what model I want

Going to have to use something like random forest because I need a multi-label classifier, or I can use sklearn.multioutput.MultiOutputClassifier and use any classifier

I think all of the data for all 12 leads is the set of features for each sample

**Define Models**

In [30]:
#TODO: Tune the hyperparameters
class RF_Model:
    def __init__(self, verbose = 1):
        self.model = RandomForestClassifier(verbose=verbose)

    def train(self, X: np.ndarray, y: np.ndarray):
        self.model.fit(X,y)

    def predict(self, X):
        return self.model.predict(X)

class LR_model:
    def __init__(self, verbose = 1):
        self.model = MultiOutputClassifier(LogisticRegression(verbose=verbose))

    def train(self, X: np.ndarray, y: np.ndarray):
        self.model.fit(X, y)

    def predict(self, X: np.ndarray):
        return self.model.predict(X)

**Data Fromatting**

PCA for the simplified models

In [31]:
class PCA_Transform:
    def __init__(self, r:int):
        self.PCA_instance = PCA(n_components=r)

    def _processData(self, X: np.ndarray):
        self.preprocess = StandardScaler()
        self.preprocess.fit(X)

    def FitData(self, X: np.ndarray):
        self._processData(X)
        self.PCA_instance.fit(self.preprocess.transform(X))

    def TransformData(self, X_train: np.ndarray, X_test: np.ndarray):
        X_train = self.preprocess.transform(X_train)
        X_test = self.preprocess.transform(X_test)
        return self.PCA_instance.transform(X_train), self.PCA_instance.transform(X_test)

## K-Fold

K-fold procedure for validation of the models

They use a validation set of 2% so something to think about

They didn't round for the outputs, seems to be a threshold in which they consider it to occur

They used precision-recall curves for things, but in total found precision, recall, specificity and F1 score

**Process Outputs**

Function for processing the output probabilities from the CNNs

Need to find the optimal thresholds for each test set before running the processing

In [32]:
def FindThresholds(y_pred: np.ndarray, y_true: np.ndarray):
    '''Optimize the thresholds based on the F1-score for the model 1 in the test set'''

    start = 0.05
    end = 0.8
    step = 0.001
    threshold_options = np.linspace(start,end,int((end-start)/step))
    optimal_thresholds = np.array([])

    for ii in range(y_pred.shape[1]):
        #use the np greater for a list of thresholds and then find max F1-score and index it back to the thresholds
        F1_scores = [f1_score(y_true=y_true[:,ii], y_pred=ProcessOutputs(y_pred[:,ii], threshold)) for threshold in threshold_options]
        optimal_thresholds = np.append(optimal_thresholds, threshold_options[np.argmax(F1_scores)])

    return optimal_thresholds


def ProcessOutputs(outputs: np.ndarray, thresholds: np.ndarray):

    threshold_check = np.array([np.greater_equal(sample, thresholds) for sample in outputs])
    threshold_outputs = threshold_check.astype(int)
    
    return threshold_outputs

**Metrics Function**

Making a function to be able to call all of the metrics each fold

In [33]:
def Score_initator():

        scores = dict()
        metrics = ['Precision', 'Recall', 'Specificity', 'F1']
        models = ['m1', 'm2', 'm3', 'm4']
        for ii, metric in enumerate(metrics):
                scores[metric] = dict()
                for jj, model in enumerate(models):
                        scores[metric][model] = dict()
                        for zz, abnomality in enumerate(abnormalities):
                                scores[metric][model][abnomality] = np.array([])
        
        return scores

def specificity_score(y_true, y_pred):
    m = confusion_matrix(y_true, y_pred, labels=[0, 1])
    spc = m[0, 0] * 1.0 / (m[0, 0] + m[0, 1])
    return spc

def Find_metrics(scores: dict, model_name: str, y_true: np.ndarray, y_pred: np.ndarray):
        
        def _findProperScore(metric, y_true, y_pred):
                if metric == 'Precision':
                        return precision_score(y_true, y_pred)  
                elif metric == 'Recall':
                        return recall_score(y_true, y_pred)
                elif metric == 'Specificity':
                        return specificity_score(y_true, y_pred)
                else:
                        return f1_score(y_true, y_pred)

        for jj, metric in enumerate(scores.keys()):
                for ii, cardio_class in enumerate(scores[metric][model_name].keys()):

                        scores[metric][model_name][cardio_class] = np.append(scores[metric][model_name][cardio_class], \
                                _findProperScore(metric, y_true[:,ii], y_pred[:,ii]))

        return scores

In [ ]:
kf = KFold(n_splits=3, shuffle=True)

train_scores = Score_initator()

#Initilaize the models that need to be trained
model_verbose = 1
model_2 = MyCNN(loss, opt, verbose = model_verbose)
'''model_3 = RF_Model(verbose = model_verbose)
model_4 = LR_model(verbose = model_verbose)'''

#PCA initlization
#PCA_transformer = PCA_Transform(r = 10)

#TODO: there is no regularization in the layers so maybe can add that
for train_index, test_index in kf.split(X = tracings_train[:,1,1], y = labels_train):

        X_train, X_test = tracings_train[train_index,:,:], tracings_train[test_index,:,:]
        y_train, y_test = labels_train[train_index], labels_train[test_index]

        #Put data in sequence for models 1 and 2 (CNN)
        train_seq, val_seq = ECGSequence.get_train_and_val(X_train, y_train, batch_size=64)
        test_seq = ECGSequence(X_test, y_test, batch_size=64)

        #Transform data with PCA for models 3 and 4
        '''for ii in range(X_train.shape[-1]):
                PCA_transformer.FitData(X_train[:,:,ii])
                PCA_X_train_temp, PCA_X_test_temp = PCA_transformer.\
                        TransformData(X_train[:,:,ii], X_test[:,:,ii])
                if ii == 0:
                        PCA_X_train = PCA_X_train_temp
                        PCA_X_test = PCA_X_test_temp
                else:
                        PCA_X_train = np.append(PCA_X_train, PCA_X_train_temp, axis = 1)
                        PCA_X_test = np.append(PCA_X_test, PCA_X_test_temp, axis = 1)'''

        #Train models
        print('\n-------------------Training model 2----------------------')
        model_2.train(train_seq, val_seq=val_seq)
        '''print('\n-------------------Training model 3----------------------')
        model_3.train(X = PCA_X_train, y = y_train)
        print('\n-------------------Training model 4----------------------')
        model_4.train(X = PCA_X_train, y = y_train)'''

        #Predict with the models - need to find optimal thresholds first for this test set
        print('\n-------------------Testing model 1----------------------')
        optimal_thresholds = FindThresholds(model_1.predict(test_seq, verbose=1), y_test)
        m1_pred = ProcessOutputs(model_1.predict(test_seq, verbose=1), optimal_thresholds)
        print('\n-------------------Testing model 2----------------------')
        m2_pred = ProcessOutputs(model_2.predict(test_seq), optimal_thresholds)
        print('\n-------------------Testing model 3----------------------')
        #m3_pred = model_3.predict(PCA_X_train)
        print('\n-------------------Testing model 4----------------------')
        #m4_pred = model_4.predict(PCA_X_train)

        #Find scores
        print('\n-------------------Finding scores----------------------')
        train_scores = Find_metrics(train_scores, 'm1', y_test, m1_pred)
        train_scores = Find_metrics(train_scores, 'm2', y_test, m2_pred)
        '''train_scores = Find_metrics(train_scores, 'm3', y_test, m3_pred)
        train_scores = Find_metrics(train_scores, 'm4', y_test, m4_pred)'''


save_path = cwd + '\\MyOutputs\\TrainScores.pickle'
SaveObject(train_scores, save_path)



In [ ]:
print(m2_pred)

## Testing (ADD PCA Calls)

**Train Models**

Train all the models that need to be trained on the entire training set and then save them

In [ ]:
save_path_1 = cwd + '\\MyOutputs\\Model1.pickle'
save_path_2 = cwd + '\\MyOutputs\\Model2.pickle'
save_path_3 = cwd + '\\MyOutputs\\Model3.pickle'
save_path_4 = cwd + '\\MyOutputs\\Model4.pickle'

train_seq, val_seq = ECGSequence.get_train_and_val(tracings_train, labels_train, batch_size=64)
#Train models
#Here the validation seq is just the test seq since we are using a k-fold analysis
print('\n-------------------Training model 2----------------------')
try:
    model_2 = LoadObject(save_path_2)
except:
    #Have to load a new model since we want to save the best one here
    model_2 = MyCNN(loss, opt, verbose = model_verbose, save_path=cwd+'\\MyOutputs\\Model2.hdf5')
    model_2.train(train_seq, val_seq=val_seq)
'''print('\n-------------------Training model 3----------------------')
try:
    model_3 = LoadObject(save_path_3)
except:
    model_3.train(X = PCA_X_train, y = y_train)
print('\n-------------------Training model 4----------------------')
try:
    model_4 = LoadObject(save_path_4)
except:
    model_4.train(X = PCA_X_train, y = y_train)'''

#Save the models
SaveObject(model_1, save_path_1)
SaveObject(model_2, save_path_2)
'''SaveObject(model_3, save_path_3)
SaveObject(model_4, save_path_4)'''

**Test Models**

Test all the models with the test set

In [ ]:
test_scores = Score_initator()
test_seq = ECGSequence(tracings_train, labels_train, batch_size=64)

#Predict with the models
print('\n-------------------Testing model 1----------------------')
optimal_thresholds = FindThresholds(model_1.predict(test_seq, verbose=1), labels_train)
m1_pred = ProcessOutputs(model_1.predict(test_seq, verbose=1), optimal_thresholds)
print('\n-------------------Testing model 2----------------------')
m2_pred = ProcessOutputs(model_2.predict(test_seq), optimal_thresholds)
print('\n-------------------Testing model 3----------------------')
#m3_pred = model_3.predict(PCA_X_test)
print('\n-------------------Testing model 4----------------------')
#m4_pred = model_4.predict(PCA_X_test)



test_scores = Find_metrics(test_scores, 'm1', labels_train, m1_pred)
#test_scores = Find_metrics(test_scores, 'm2', labels_test, m2_pred)
save_path = cwd + '\\MyOutputs\\TestScores.pickle'
SaveObject(test_scores, save_path)
print(test_scores)